# intro 

*slack message Paolo*:

OK in order to be helpful I needed to take a look myself.

I suggest structuring the work in these parts:

1. Understand the generative process (no real data, just generation based on your mental model of thigs). I did this in the script attached 
    (my plots look different from yours)
    
2. Make sure your data follow the hypothesized model. specifically, in the considered model you assume that the data come from beta_binomial, where n is linear in age, while mean_beta and var_beta do not depend on age. Is this true? You can check it by (i) plot coverage vs age (is this linear?); (ii) fit beta binomials on young and old people groups separately and see how the MLEs of mean_beta and var_beta correlate across genes.
    
3. If this looks good -> your model is reasonable -> you can assume things are fine. If you see deviations you need to rethinkg about the generative processs---e.g., may var_beta also depend on age?


*My thoughts on this:*

1. Absolutely, and first and foremost thanks a lot for the clarity and advice. Indeed, I could reproduce data and plots which looked like yours. The difference is, as you've already pointed out below, if one makes the variance of the Beta distribution dependent on coverage or not. In the code you've sent, the were not linked. Below they are, as well as in all the scenarios I showed in previous meetings. 

2. I went back to the original GTEx heteroplasmy data again and found this was not super clear. I have some slides to show this if you're interested, but in brief: (i) there seems to be some modest relationship between coverage and age overall. And it's in the ball park of being linear. (ii) for the positions the linear model seemed to pick up on when testing for relationship between heteroplasmy and donor age, the relationship looked linear as well (analytical p-values; study-wide bonferroni). (iii) for the same postions, there definitely seems to be an inverse relationship between the variance in heteroplasmy estimates and coverage. *In conclusion, this indicates to me, that we should link coverage with variance in the Beta distribution, which is what I did below.* Age vs. coverage may not be linear relationship in the real data. But as long as it induces an inverse relationship between variance in heteroplasmy and coverage which is also correlated with age, this seems fair to me.

3. I didn't manage to fit the models yet. This is ongoing and I'll update you on that.

# code
## init

In [ ]:
# select python environment `gene_tools`
# source deps
import numpy as np
import pdb
import pylab as plt
import pandas as pd
import os
import random
import session_info
from matplotlib.backends.backend_pdf import PdfPages
from scipy.stats import truncnorm

## define functions

In [ ]:
# 2024_01_31: let's turn interactive code into a function
def sample_cohort_specs(n_donors = 500, mean_age = 50, sd_age = 10, 
                        min_age_bound = 20, max_age_bound = 70, 
                        min_coverage = 10,  r_sqr = 0.1, 
                        slope = 0.1, mean_coverage=1000):
    """
    Generate synthetic cohort similar to GTEx data. The main point is to simulate
    coverage values which are linearly dependent on donor age. The parameter 
    values are chosen such that the properties of the simulated cohort match
    those of GTEx tissues best. These simulations are best thought of as "per 
    tissue, per position". 

    - donor ages are simulated using a truncated normal. 
    - coverage values are simulated using a linear model and Gaussian noise.
    

    n_donors:               number of donors to simulate 
    mean_age:               mean of the resulting age distribution
    sd_age:                 standard deviation of the resulting age distribution
    min_age_bound:          minimum of donor age distribution to simulate
    max_age_bound:          minimum of donor age distribution to simulate
    slope:                  slope of age coverage linear relationship.  
    r_sqr:                  R^2 value specified for age ~ coverage relationship.
    mean_coverage:          mean of coverage values to aim for when simulating.
    min_coverage:           minimum coverage value found in real data.

    
    returns:   "synthetic cohort"  for tissue X and one position. Object is PD
               containing columns age, mean_age, coverage, mean_coverage, coverage.

    """

    # simulate cohort ----------------------------------------------------------
    # init empty data frame
    sim_df = pd.DataFrame(np.zeros(n_donors),columns=["donor_id"])
    # generate donor_id 
    sim_df['donor_id'] = 'donor_' + (sim_df.index + 1).astype(str)


    # simulate donor ages ------------------------------------------------------
    # calculate the truncated normal bounds
    a = (min_age_bound - mean_age) / sd_age
    b = (max_age_bound - mean_age) / sd_age
    # generate ages from a truncated normal distribution
    ages = truncnorm.rvs(a, b, loc = mean_age, scale = sd_age, size = n_donors)
    # round ages to integers
    simulated_age = np.round(ages).astype(int)


    # simulate coverage in linear dependence of donor age ----------------------
    # the key question here is how to relate mean_coverage to our linear model
    # slope coverage is defined as the [%] of change per year of life
    # delete:  slope_coverage = mean_coverage * slope_gradient 
    # slope is fixed and intercept needs to be computed
    #     beta = simulated_age * slope_coverage
    intercept = mean_coverage - np.mean(simulated_age) * slope
    var_coverage = (((1 - r_sqr)/r_sqr) * slope**2) * np.var(simulated_age)
    sd_coverage = np.sqrt(var_coverage)
    simulated_coverage = intercept + simulated_age * slope + np.random.normal(loc = 0, scale = sd_coverage, size = len(sim_df)) 
    
    
    # for coverage < min_coverage repeat simulation  
    simulated_coverage = np.maximum(simulated_coverage, min_coverage)  
    # continue sampling if there are still values smaller than min_coverage
    while np.any(simulated_coverage <= min_coverage):
        # resample only the values that are smaller than min_coverage
        resample_mask = simulated_coverage <= min_coverage
        resample_size = np.sum(resample_mask)
        # generate new values for the resampled subset
        new_values = intercept + simulated_age[resample_mask] * slope + np.random.normal(loc = 0, scale = sd_coverage, size = resample_size)
        # update the simulated_coverage array with the new values
        simulated_coverage[resample_mask] = np.maximum(new_values, min_coverage)

    # add return values to synthetic cohorts -----------------------------------
    sim_df["age"] = simulated_age
    sim_df["mean_age"] = np.mean(sim_df["age"])
    sim_df["coverage_non_rounded"] = simulated_coverage
    sim_df["mean_coverage"] = np.mean(sim_df["coverage_non_rounded"])
    sim_df["coverage"] = np.round(simulated_coverage).astype(int)


    # return result ------------------------------------------------------------
    return sim_df    



In [ ]:
# 2024_01_30: cleanup
def sample_beta_binomial_fixed_variance(sim_df, mean_beta=0.5, var_beta=0.1):
    """
    uses simulated coverage values from linear age ~ coverage relationship and 
    draw synthetic heteroplasmy values using a BetaBinomial only dependent on 
    coverage.

    mean_beta:  mean to be used in the Beta function.
    var_beta:   variance to be used in the Beta function.
    
    returns:    synthetic heteroplasmy counts/levels attached as cols to input 
                synthetic cohort PD (generated 1 step above).

    """
    # use constant coverage
    sim_df["var_beta"] = var_beta

    # set mean to expected mean_het
    sim_df["mean_het"] = mean_beta

    # calculate alpha and beta from mean and variance
    sim_df["common_factor"] = sim_df["mean_het"] * (1 - sim_df["mean_het"]) / sim_df["var_beta"] - 1
    sim_df["alpha"] = sim_df["mean_het"] * sim_df["common_factor"]
    sim_df["beta"] = (1 - sim_df["mean_het"]) * sim_df["common_factor"]

    # 2024_01_01: adding pseudocount to ensure alpha and beta are positive
    sim_df["alpha"] = np.maximum(sim_df["alpha"], 1e-6)  # Set a minimum value, adjust as needed
    sim_df["beta"] = np.maximum(sim_df["beta"], 1e-6)

    # sample from p_beta from Beta for using it in Bernoulli draw below
    sim_df["p_beta"] = np.random.beta(sim_df["alpha"],sim_df["beta"])

    # sample from the Binomial distribution using the sampled Beta value
    sim_df["het_counts"] = np.random.binomial(sim_df["coverage"], sim_df["p_beta"])
    sim_df["het_level"] = sim_df["het_counts"]/sim_df["coverage"]

    return sim_df       

## apply functions -- manual for testing

In [ ]:
# set params manually for debugging before running automated pipeline above
## age ~ coverage params
n_donors = 500
mean_age = 50
sd_age = 15
min_age_bound = 20
max_age_bound = 70
min_coverage = 10

# alternate to find spot from graphs which combinations are plausible
r_sqr = 0.1
slope = 350
mean_coverage = 20000
## heteroplasmy params
mean_beta = 0.5
var_beta = 0.01

## set N positions to simulate
n_pos = 100

# init res list
sim_list = []
# loop through the number of positions you want to simulate
for pos in range(1, n_pos + 1):
    # simulate coverage
    tmp_df = sample_cohort_specs(n_donors=n_donors,
                                 mean_age=mean_age,
                                 sd_age=sd_age,
                                 min_age_bound=min_age_bound,
                                 max_age_bound=max_age_bound,
                                 min_coverage=min_coverage,
                                 r_sqr=r_sqr,
                                 slope=slope, 
                                 mean_coverage=mean_coverage) 
    # simulate heteroplasmy inversely correlated with coverage using beta binomial
    tmp_df = sample_beta_binomial_fixed_variance(sim_df=tmp_df, 
                                                 mean_beta=mean_beta, 
                                                 var_beta=var_beta)
    # add Pos column 
    tmp_df["Pos"] = pos
    # append the result to the list
    sim_list.append(tmp_df)

# concatenate the results into a single DataFrame
sim_df = pd.concat(sim_list, ignore_index=True)

# show contents sim_df 
#sim_df

In [ ]:
# plot mean_het level
mean_het_level = np.mean(sim_df["het_level"])

# make plots for sanity checking 
plt.figure(1, figsize=(18, 5))

# # 1
plt.subplot(131)
plt.plot(sim_df["age"], sim_df["coverage"], '.k')
plt.title('simulations: age vs. coverage \n', fontsize = 16, fontweight = "bold")
plt.xlabel('age [y]', fontsize = 14, fontweight = "bold")
plt.ylabel('coverage [reads]', fontsize = 14, fontweight = "bold")

# # 2
plt.subplot(132)
plt.plot(sim_df["age"], sim_df["het_level"], '.k')
plt.axhline(y=mean_het_level, color='r', linestyle='-', label=f'Mean Het Level: {mean_het_level:.2f}')
plt.title(f'simulations: coverage vs. heteroplasmy \n(Mean het_level simulated: {mean_het_level:.5f})', fontsize = 16, fontweight = "bold")
#plt.title('simulations: age vs. heteroplasmy', fontsize = 16, fontweight = "bold")
plt.ylabel('heteroplasmy [%]', fontsize = 14, fontweight = "bold")
plt.xlabel('age [y]', fontsize = 14, fontweight = "bold")
plt.ylim(0, 1)

# # 3
plt.subplot(133)
plt.plot(sim_df["coverage"], sim_df["het_level"], '.k')
plt.axhline(y=mean_het_level, color='r', linestyle='-', label=f'Mean Het Level: {mean_het_level:.2f}')
plt.ylabel('heteteroplasmy [%]', fontsize = 14, fontweight = "bold")
plt.xlabel('coverage [reads]', fontsize = 14, fontweight = "bold")
#plt.title('simulations: coverage vs. heteroplasmy', fontsize = 16, fontweight = "bold")
plt.title(f'simulations: coverage vs. heteroplasmy \n(Mean heat_level simulated: {mean_het_level:.5f})', fontsize = 16, fontweight = "bold")
plt.ylim(0, 1)
plt.tight_layout()
plt.show()

In [ ]:
sim_df["het_level"].mean()

In [ ]:
# Assuming sim_df is a pandas DataFrame
correlation_matrix = np.corrcoef(sim_df["coverage_non_rounded"], sim_df["age"])

# The correlation coefficient is at position (0, 1) or (1, 0) in the matrix
correlation_coefficient = correlation_matrix[0, 1]

print("Correlation Coefficient:", correlation_coefficient)
print("sqrt: Correlation Coefficient:", np.sqrt(correlation_coefficient))

## apply functions -- manual and motivated by observed but extreme data values

### read in observed but extreme values for simulating 8 cohorts


In [ ]:
params_df = pd.read_excel("/Users/simon.wengert/git/mtDNA_variants/metadata/params/2024_01_30_het_sim_params.xlsx")
params_df = params_df[params_df['run'] == True]
params_df["n_donors"] = params_df["n_donors"].astype(int)
# params_df.insert(4, 'simulation', range(1, len(params_df) + 1))

params_df

### for each of these cohorts, assess resulting simulated heterplasmy levels when changing `mean_beta` and `var_beta`

In [ ]:
# Create an empty list to store simulated data frames
simulated_dfs = []
n_pos = 100
# Create a PDF to save all the plots
params_df.iloc[0]
# Retrieve params
row_id = row["row_id"]
change_param = row["change_param"]
step_param = row["step_param"]
n_donors = row['n_donors']
mean_age = row["mean_age"]
sd_age = row["sd_age"]
min_age_bound = row["min_age_bound"]
max_age_bound = row["max_age_bound"]
min_coverage = row["min_coverage"]
slope = row["slope"]
r_sqr = row["r_sqr"]
mean_coverage = row["mean_coverage"]


## heteroplasmy params
mean_beta = 0.025
var_beta = 0.000001

## set N positions to simulate
n_pos = 100

# init res list
sim_list = []
# loop through the number of positions you want to simulate
for pos in range(1, n_pos + 1):
    # simulate coverage
    tmp_df = sample_cohort_specs(n_donors=n_donors,
                                 mean_age=mean_age,
                                 sd_age=sd_age,
                                 min_age_bound=min_age_bound,
                                 max_age_bound=max_age_bound,
                                 min_coverage=min_coverage,
                                 r_sqr=r_sqr,
                                 slope=slope, 
                                 mean_coverage=mean_coverage) 
    # simulate heteroplasmy inversely correlated with coverage using beta binomial
    tmp_df = sample_beta_binomial_fixed_variance(sim_df=tmp_df, 
                                                 mean_beta=mean_beta, 
                                                 var_beta=var_beta)
    # add Pos column 
    tmp_df["Pos"] = pos
    # append the result to the list
    sim_list.append(tmp_df)

# concatenate the results into a single DataFrame
sim_df = pd.concat(sim_list, ignore_index=True)


sim_df

In [ ]:
# 0 based index which means the thing below is XXX
row = params_df.iloc[2] 
row

In [ ]:
# Create an empty list to store simulated data frames
simulated_dfs = []
n_pos = 100
# Create a PDF to save all the plots

# Retrieve params
row_id = row["row_id"]
change_param = row["change_param"]
step_param = row["step_param"]
n_donors = row['n_donors']
mean_age = row["mean_age"]
sd_age = row["sd_age"]
min_age_bound = row["min_age_bound"]
max_age_bound = row["max_age_bound"]
min_coverage = row["min_coverage"]
slope = row["slope"]
r_sqr = row["r_sqr"]
mean_coverage = row["mean_coverage"]

# Function to calculate mean heteroplasmy for given mean_beta and var_beta
def calculate_mean_het(mean_beta, var_beta):
    sim_list = []
    for pos in range(1, n_pos + 1):
        tmp_df = sample_cohort_specs(n_donors=n_donors,
                                     mean_age=mean_age,
                                     sd_age=sd_age,
                                     min_age_bound=min_age_bound,
                                     max_age_bound=max_age_bound,
                                     min_coverage=min_coverage,
                                     r_sqr=r_sqr,
                                     slope=slope,
                                     mean_coverage=mean_coverage)
        tmp_df = sample_beta_binomial_fixed_variance(sim_df=tmp_df,
                                                     mean_beta=mean_beta,
                                                     var_beta=var_beta)
        tmp_df["Pos"] = pos
        sim_list.append(tmp_df)

    sim_df = pd.concat(sim_list, ignore_index=True)
    return sim_df["het_level"].mean()


# init grid for comparison:
mean_beta_values = [0.0, 0.01, 0.02, 0.05, 0.1, 0.2, 0.3, 0.4,  0.5, 0.75, 0.9]
# var_beta_values = [0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1,1.0,10]
var_beta_values = [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1]



# Create an empty dictionary to store results
result_dict = {}

# Iterate over the grid
for mean_beta in mean_beta_values:
    for var_beta in var_beta_values:
        # Calculate het_level
        current_het_level = calculate_mean_het(mean_beta, var_beta)
        
        # Store the result in the dictionary
        result_dict[(mean_beta, var_beta)] = current_het_level

# Print the results
for (mean_beta, var_beta), het_level in result_dict.items():
    print(f'Mean Beta: {mean_beta}, Var Beta: {var_beta}, Het Level: {het_level}')


In [ ]:
# make heatmap
import seaborn as sns
import matplotlib.pyplot as plt

# Convert the result_dict to a DataFrame for easier plotting
result_df = pd.DataFrame(list(result_dict.items()), columns=['Beta', 'Het_Level'])

# Split the 'Beta' column into two separate columns: 'Mean_Beta' and 'Var_Beta'
result_df[['Mean_Beta', 'Var_Beta']] = pd.DataFrame(result_df['Beta'].tolist(), index=result_df.index)

# Create a pivot table for plotting the heatmap
heatmap_data = result_df.pivot_table(index='Var_Beta', columns='Mean_Beta', values='Het_Level')

# Plot the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(heatmap_data, cmap='viridis', annot=True, fmt=".4f", cbar_kws={'label': 'Het Level'})
plt.title('Het Level for Mean Beta vs Var Beta')
plt.xlabel('Mean Beta')
plt.ylabel('Var Beta')
plt.show()


In [ ]:
# Convert the result_dict to a DataFrame for easier plotting
result_df = pd.DataFrame(list(result_dict.items()), columns=['Beta', 'Het_Level'])

# Split the 'Beta' column into two separate columns: 'Mean_Beta' and 'Var_Beta'
result_df[['Mean_Beta', 'Var_Beta']] = pd.DataFrame(result_df['Beta'].tolist(), index=result_df.index)

# Plot a scatter plot with log scale on both axes, larger dots, and legend outside the plot
plt.figure(figsize=(12, 8))
#scatter_plot = sns.scatterplot(data=result_df, x='Mean_Beta', y='Var_Beta', hue='Het_Level', palette='viridis', s=500)
scatter_plot = sns.scatterplot(data=result_df, x='Mean_Beta', y='Var_Beta', hue='Het_Level', palette='viridis', s=700, marker='s')
plt.xscale('log')
plt.yscale('log')
plt.title('Het Level for Mean Beta vs Var Beta (log scale)', fontsize=16, fontweight='bold')
plt.xlabel('Mean Beta (log scale)', fontsize=14, fontweight='bold')
plt.ylabel('Var Beta (log scale)', fontsize=14, fontweight='bold')
plt.legend(title='Het Level', bbox_to_anchor=(1.05, 1), loc='upper left', fontsize='large', title_fontsize='large')

# Increase font size and boldness for axis labels
scatter_plot.set_xticklabels(scatter_plot.get_xticks(), fontsize=12, fontweight='bold')
scatter_plot.set_yticklabels(scatter_plot.get_yticks(), fontsize=12, fontweight='bold')

plt.show()


In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

# Function to calculate mean heteroplasmy for given mean_beta and var_beta
def calculate_mean_het(mean_beta, var_beta):
    sim_list = []
    for pos in range(1, n_pos + 1):
        tmp_df = sample_cohort_specs(n_donors=n_donors,
                                     mean_age=mean_age,
                                     sd_age=sd_age,
                                     min_age_bound=min_age_bound,
                                     max_age_bound=max_age_bound,
                                     min_coverage=min_coverage,
                                     r_sqr=r_sqr,
                                     slope=slope,
                                     mean_coverage=mean_coverage)
        tmp_df = sample_beta_binomial_fixed_variance(sim_df=tmp_df,
                                                     mean_beta=mean_beta,
                                                     var_beta=var_beta)
        tmp_df["Pos"] = pos
        sim_list.append(tmp_df)

    sim_df = pd.concat(sim_list, ignore_index=True)
    return sim_df["het_level"].mean()

# Specify your desired mean heteroplasmy objective
mean_het_objective = 0.7  # Replace this with your desired value

# Set a tolerance level for the mean_het_objective
tolerance = 0.1

# Initialize lists to store optimization history
mean_beta_history = []
var_beta_history = []
mean_het_history = []

# Define the grid of mean_beta and var_beta values
mean_beta_values = np.arange(0.00, 1.05, 0.05)
mean_beta_values = [0.0, 0.1, 0.2, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
var_beta_values = [0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1]

# Iterate over the grid
for mean_beta in mean_beta_values:
    for var_beta in var_beta_values:
        current_mean_beta = mean_beta
        current_var_beta = var_beta

        # Initialize values
        current_mean_het = calculate_mean_het(current_mean_beta, current_var_beta)

        # Print statements for debugging
        print(f"Starting optimization for mean_beta={mean_beta}, var_beta={var_beta}")

        # Iterate until the mean_het_objective is reached
        while np.abs(current_mean_het - mean_het_objective) > tolerance:
            # Adjust mean_beta and var_beta
            current_mean_beta += 0.05  # You can adjust the step size as needed
            current_var_beta += 0.00001  # You can adjust the step size as needed

            # Recalculate mean_het
            current_mean_het = calculate_mean_het(current_mean_beta, current_var_beta)
            print(f"Current mean_het: {current_mean_het}")

        # Store the optimization results
        mean_beta_history.append(current_mean_beta)
        var_beta_history.append(current_var_beta)
        mean_het_history.append(current_mean_het)

# Plot the contour plot
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Create a grid of mean_beta and var_beta values
mean_beta_grid, var_beta_grid = np.meshgrid(mean_beta_values, var_beta_values)

# Calculate mean_het for each combination of mean_beta and var_beta
mean_het_grid = np.array([[calculate_mean_het(mean_b, var_b) for mean_b in mean_beta_values] for var_b in var_beta_values])

# Plot the contour plot
ax.contour(mean_beta_grid, var_beta_grid, mean_het_grid, levels=20, cmap='viridis', label='Mean Heteroplasmy')

# Plot the optimization path
ax.scatter(mean_beta_history, var_beta_history, mean_het_history, color='red', marker='o', label='Optimization Path')

ax.set_xlabel('Mean Beta')
ax.set_ylabel('Var Beta')
ax.set_zlabel('Mean Heteroplasmy')
ax.set_title('Optimization Path for Mean Heteroplasmy')

# Show the plot
plt.show()


In [ ]:
# make plots for sanity checking 
plt.figure(1, figsize=(18, 5))

# # 1
plt.subplot(131)
plt.plot(sim_df["age"], sim_df["coverage"], '.k')
plt.title('simulations: age vs. coverage', fontsize = 16, fontweight = "bold")
plt.xlabel('age [y]', fontsize = 14, fontweight = "bold")
plt.ylabel('coverage [reads]', fontsize = 14, fontweight = "bold")

# # 2
plt.subplot(132)
plt.plot(sim_df["age"], sim_df["het_level"], '.k')
plt.title('simulations: age vs. heteroplasmy', fontsize = 16, fontweight = "bold")
plt.ylabel('heteroplasmy [%]', fontsize = 14, fontweight = "bold")
plt.xlabel('age [y]', fontsize = 14, fontweight = "bold")
plt.ylim(0, 1)

# # 3
plt.subplot(133)
plt.plot(sim_df["coverage"], sim_df["het_level"], '.k')
plt.ylabel('heteteroplasmy [%]', fontsize = 14, fontweight = "bold")
plt.xlabel('coverage [reads]', fontsize = 14, fontweight = "bold")
plt.title('simulations: coverage vs. heteroplasmy', fontsize = 16, fontweight = "bold")
plt.ylim(0, 1)
plt.tight_layout()
plt.show()

## apply functions -- automated for production

### simulate heteroplasmy values for selected params

In [ ]:
params_df = pd.read_excel("/Users/simon.wengert/git/mtDNA_variants/metadata/params/2024_01_30_het_sim_params.xlsx")
params_df = params_df[params_df['run'] == True]
params_df["n_donors"] = params_df["n_donors"].astype(int)
params_df
#params_df.drop(['change_param', 'step_param'])

In [ ]:
# run simulations as specified in params_df
save_path = "/Users/simon.wengert/data/mtDNA_variants/metadata/cohort_files/gtex_v8/"

# Create an empty list to store simulated data frames
simulated_dfs = []
n_pos = 1000
# Create a PDF to save all the plots
date = "2024_05_01_"
max_row = max(params_df["row_id"].astype(int))
for index, row in params_df.iterrows():
    # tell me where we are:
    print(f"currently simulating row: {index} out of {max_row}")
    # Retrieve params
    row_id = row["row_id"]
    change_param = row["change_param"]
    step_param = row["step_param"]
    n_donors = row['n_donors']
    mean_age = row["mean_age"]
    sd_age = row["sd_age"]
    min_age_bound = row["min_age_bound"]
    max_age_bound = row["max_age_bound"]
    min_coverage = row["min_coverage"]
    slope = row["slope"]
    r_sqr = row["r_sqr"]
    mean_coverage = row["mean_coverage"]
    mean_beta = row["mean_beta"]
    var_beta = row["var_beta"]
    

    #try:    
    # init res list
    sim_list = []
    # loop through the number of positions you want to simulate
    for pos in range(1, n_pos + 1):
        # simulate coverage
        tmp_df = sample_cohort_specs(n_donors=n_donors, 
                                     mean_age=mean_age,
                                     sd_age=sd_age,
                                     min_age_bound=min_age_bound,
                                     max_age_bound=max_age_bound,
                                     min_coverage=min_coverage,
                                     slope=slope,
                                     r_sqr=r_sqr,
                                     mean_coverage=mean_coverage)
        # simulate heteroplasmy inversely correlated with coverage using beta binomial
        tmp_df = sample_beta_binomial_fixed_variance(sim_df=tmp_df, 
                                                     mean_beta=mean_beta, 
                                                     var_beta=var_beta)
        # add Pos column 
        tmp_df["Pos"] = pos
        # append the result to the list
        sim_list.append(tmp_df)

    # concatenate the results into a single DataFrame
    sim_df = pd.concat(sim_list, ignore_index=True)
    
    # Add param record to sim_df
    sim_df["row_id"] = row_id
    sim_df["date"] = date
    sim_df["change_param"] = change_param
    sim_df["step_param"] = step_param
    sim_df["n_donors"] = n_donors
    sim_df["mean_age"] = mean_age
    sim_df["sd_age"] = sd_age
    sim_df["min_age_bound"] = min_age_bound
    sim_df["max_age_bound"] = max_age_bound
    sim_df["min_coverage"] = min_coverage
    sim_df["slope"] = slope
    sim_df["r_sqr"] = r_sqr
    sim_df["mean_coverage"] = mean_coverage
    sim_df["mean_beta"] = mean_beta
    sim_df["var_beta"] = var_beta

    #except Exception as e:
    #   # Handle the exception (you can print an error message or log the error)
    #    print(f"Error in simulation for params: {row}")
    #    print(f"Error message: {e}")
    #    continue  # Continue to the next iteration if an error occurs
    
    # Append the simulated DataFrame to the list
    simulated_dfs.append(sim_df)

# Merge all simulated data frames into one
print(f"concatenating simulations df --- ")   
merged_sim_df = pd.concat(simulated_dfs)

# Save the merged simulated DataFrame to disk
print(f"writing file to disk --- ") 
# merged_sim_df.to_csv(os.path.join(save_path, f"{date}merged_simulated_data.tsv"), sep="\t", index=False)#
merged_sim_df.to_csv(os.path.join(save_path, f"{date}merged_simulated_data_var_beta_large.tsv"), sep="\t", index=False)

# scratch ----------------------------------------------------------------

In [ ]:
# Load parameters from the text file into a DataFrame
params_df = pd.read_csv("/Users/simon.wengert/git/mtDNA_variants/metadata/utils/het_sim_params_df_2.txt", sep="\t")

# Define absolute paths for saving files
save_path = "/Users/simon.wengert/data/mtDNA_variants/metadata/cohort_files/gtex_v8/"
figure_path = "/Users/simon.wengert/git/mtDNA_variants/results/3_pheno_test/simulations/"

# Create an empty list to store simulated data frames
simulated_dfs = []

# Create a PDF to save all the plots
date = "2023_12_13_"
pdf_path = os.path.join(figure_path, f"{date}heteroplasmy_simulation_plots_combined.pdf")
with PdfPages(pdf_path) as pdf:
    for index, row in params_df.iterrows():
    
        # Retrieve params
        row_id = row['row_id']
        base_coverage = row['base_coverage']
        slope = row['slope']
        noise_factor = row['noise_factor']
        mean_beta = row['mean_beta']
        var_beta = row['var_beta']
        
        try:
            # Simulate coverage
            sim_df = sample_cohort_specs(base_coverage=base_coverage, slope=slope, noise_factor=noise_factor)
            
            # Simulate heteroplasmy inversely correlated with coverage using beta binomial
            sim_df = sample_beta_binomial_fixed_variance(sim_df=sim_df, mean_beta=mean_beta, var_beta=var_beta)    

            # Add param record to sim_df
            sim_df["sim_param_row_id"] = row_id
            sim_df["sim_param_date"] = date
            sim_df['sim_param_base_coverage'] = base_coverage
            sim_df['sim_param_slope'] = slope
            sim_df['sim_param_noise_factor'] = noise_factor
            sim_df['sim_param_mean_beta'] = mean_beta
            sim_df['sim_param_var_beta'] = var_beta

            # Joint title
            joint_title = f"Simulations: Base Coverage={base_coverage}, Slope={slope}, Noise Factor={noise_factor}, Mean Beta={mean_beta}, Var Beta={var_beta}"

            # Plot heteroplasmy simulations
            plt.figure(figsize=(18, 5))

            # # 1
            plt.subplot(131)
            plt.plot(sim_df["age"], sim_df["coverage"], '.k')
            plt.title('Age vs. Coverage', fontsize=16, fontweight="bold")
            plt.xlabel('Age [y]', fontsize=14, fontweight="bold")
            plt.ylabel('Coverage [reads]', fontsize=14, fontweight="bold")
            plt.suptitle(joint_title, fontsize=18, fontweight="bold")

            # # 2
            plt.subplot(132)
            plt.plot(sim_df["age"], sim_df["het_level"], '.k')
            plt.title('Age vs. Heteroplasmy', fontsize=16, fontweight="bold")
            plt.ylabel('Heteroplasmy [%]', fontsize=14, fontweight="bold")
            plt.xlabel('Age [y]', fontsize=14, fontweight="bold")
            plt.ylim(0, 1)

            # # 3
            plt.subplot(133)
            plt.plot(sim_df["coverage"], sim_df["het_level"], '.k')
            plt.ylabel('Heteroplasmy [%]', fontsize=14, fontweight="bold")
            plt.xlabel('Coverage [reads]', fontsize=14, fontweight="bold")
            plt.title('Coverage vs. Heteroplasmy', fontsize=16, fontweight="bold")
            plt.ylim(0, 1)
            plt.tight_layout()

            # Append the plot to the PDF
            pdf.savefig(dpi=80)
            plt.close()  # Close the current figure to avoid overlapping plots
        
        except Exception as e:
            # Handle the exception (you can print an error message or log the error)
            print(f"Error in simulation for params: {row}")
            print(f"Error message: {e}")
            continue  # Continue to the next iteration if an error occurs
        
        # Save the simulated DataFrame to disk
        file_name = f"{date}heteroplasmy_simulations_slope_{slope}_mean_beta_{mean_beta}_var_beta_{var_beta}.tsv"
        sim_df.to_csv(os.path.join(save_path, file_name), sep="\t", index=False)
    
        # Append the simulated DataFrame to the list
        simulated_dfs.append(sim_df)

# Merge all simulated data frames into one
merged_sim_df = pd.concat(simulated_dfs)

# Save the merged simulated DataFrame to disk
merged_sim_df.to_csv(os.path.join(save_path, f"{date}_merged_simulated_data.tsv"), sep="\t", index=False)

In [ ]:
# Load parameters from the text file into a DataFrame
params_df = pd.read_csv("/Users/simon.wengert/git/mtDNA_variants/metadata/utils/het_sim_params_df_2.txt", sep="\t")

# Define absolute paths for saving files
save_path = "/Users/simon.wengert/data/mtDNA_variants/metadata/cohort_files/gtex_v8/"
figure_path = "/Users/simon.wengert/git/mtDNA_variants/results/3_pheno_test/simulations/"

# Create an empty list to store simulated data frames
simulated_dfs = []

# Loop through each row in the parameters DataFrame
for index, row in params_df.iterrows():
    
    # Retrieve params
    date = "2023_12_13_"
    base_coverage = row['base_coverage']
    slope = row['slope']
    noise_factor = row['noise_factor']
    mean_beta = row['mean_beta']
    var_beta = row['var_beta']
    
    # Simulate coverage
    sim_df = sample_cohort_specs(base_coverage=base_coverage, slope=slope, noise_factor=noise_factor)
    
    # Simulate heteroplasmy inversely correlated with coverage using beta binomial
    sim_df = sample_beta_binomial_fixed_variance(sim_df=sim_df, mean_beta=mean_beta, var_beta=var_beta)    

    # Add param record to sim_df
    sim_df["sim_param_date"] = date
    sim_df['sim_param_base_coverage'] = base_coverage
    sim_df['sim_param_slope'] = slope
    sim_df['sim_param_noise_factor'] = noise_factor
    sim_df['sim_param_mean_beta'] = mean_beta
    sim_df['sim_param_var_beta'] = var_beta

    # Joint title
    joint_title = f"Simulations: Base Coverage={base_coverage}, Slope={slope}, Noise Factor={noise_factor}, Mean Beta={mean_beta}, Var Beta={var_beta}"

    # Plot heteroplasmy simulations
    plt.figure(figsize=(18, 5))

    # # 1
    plt.subplot(131)
    plt.plot(sim_df["age"], sim_df["coverage"], '.k')
    plt.title('Age vs. Coverage', fontsize=16, fontweight="bold")
    plt.xlabel('Age [y]', fontsize=14, fontweight="bold")
    plt.ylabel('Coverage [reads]', fontsize=14, fontweight="bold")
    plt.suptitle(joint_title, fontsize=18, fontweight="bold")

    # # 2
    plt.subplot(132)
    plt.plot(sim_df["age"], sim_df["het_level"], '.k')
    plt.title('Age vs. Heteroplasmy', fontsize=16, fontweight="bold")
    plt.ylabel('Heteroplasmy [%]', fontsize=14, fontweight="bold")
    plt.xlabel('Age [y]', fontsize=14, fontweight="bold")
    plt.ylim(0, 1)

    # # 3
    plt.subplot(133)
    plt.plot(sim_df["coverage"], sim_df["het_level"], '.k')
    plt.ylabel('Heteroplasmy [%]', fontsize=14, fontweight="bold")
    plt.xlabel('Coverage [reads]', fontsize=14, fontweight="bold")
    plt.title('Coverage vs. Heteroplasmy', fontsize=16, fontweight="bold")
    plt.ylim(0, 1)
    plt.tight_layout()

    # Save the plot to a PDF file
    plot_name = f"{date}heteroplasmy_simulation_plots_slope_{slope}_mean_beta_{mean_beta}_var_beta_{var_beta}.pdf"
    plt.savefig(os.path.join(figure_path,plot_name))
    plt.close()  # Close the current figure to avoid overlapping plots

    # Save the simulated DataFrame to disk
    file_name = f"{date}heteroplasmy_simulations_slope_{slope}_mean_beta_{mean_beta}_var_beta_{var_beta}.tsv"
    sim_df.to_csv(os.path.join(save_path,file_name), sep="\t", index=False)
    
    # Append the simulated DataFrame to the list
    simulated_dfs.append(sim_df)

# Merge all simulated data frames into one
merged_sim_df = pd.concat(simulated_dfs)

# Save the merged simulated DataFrame to disk
merged_sim_df.to_csv(os.path.join(save_path, "{date}_merged_simulated_data.tsv"), sep="\t", index=False)


## session info

In [ ]:
session_info.show()

# scratch

In [ ]:
# 2024_01_29: let's turn interactive code into a function
def sample_cohort_specs(n_donors = 250, mean_age = 50, sd_age = 10, 
                        min_age_bound = 20, max_age_bound = 90, 
                        min_coverage = 10, max_coverage = 50000,
                        slope_gradient = 0.1, noise_factor = 0.2, 
                        mean_coverage=1000):
    """
    Generate synthetic cohort similar to GTEx data. The main point is to simulate
    coverage values which are linearly dependent on donor age. The parameter 
    values are chosen such that the properties of the simulated cohort match
    those of GTEx tissues best. These simulations are best thought of as "per 
    tissue, per position". 

    - donor ages are simulated using a truncated normal. 
    - coverage values are simulated using a linear model and Gaussian noise.
    

    n_donors:               number of donors to simulate 
    mean_age:               mean of the resulting age distribution
    sd_age:                 standard deviation of the resulting age distribution
    min_age_bound:          minimum of donor age distribution to simulate
    max_age_bound:          minimum of donor age distribution to simulate
    slope_gradient:         % of change in slope per year of age. Specified by user. 
    mean_coverage:          mean of coverage values to aim for when simulating.
    min_coverage:           minimum coverage value found in real data.
    noise_factor:           % of mean_coverage defining how much gaussian noise 
                            to add to age vs. coverage line.

    
    returns:   "synthetic cohort"  for tissue X and one position. Object is PD
               containing columns age, mean_age, coverage, mean_coverage, coverage.

    """

    # simulate cohort ----------------------------------------------------------
    # init empty data frame
    sim_df = pd.DataFrame(np.zeros(n_donors),columns=["donor_id"])
    # generate donor_id 
    sim_df['donor_id'] = 'donor_' + (sim_df.index + 1).astype(str)


    # simulate donor ages ------------------------------------------------------
    # calculate the truncated normal bounds
    a = (min_age_bound - mean_age) / sd_age
    b = (max_age_bound - mean_age) / sd_age
    # generate ages from a truncated normal distribution
    ages = truncnorm.rvs(a, b, loc = mean_age, scale = sd_age, size = n_donors)
    # round ages to integers
    simulated_age = np.round(ages).astype(int)


    # simulate coverage in linear dependence of donor age ----------------------
    # the key question here is how to relate mean_coverage to our linear model
    # slope coverage is defined as the [%] of change per year of life
    slope_coverage = mean_coverage * slope_gradient 
    noise_coverage = mean_coverage * noise_factor
    simulated_coverage = simulated_age * slope_coverage + np.random.normal(loc=0, scale=noise_coverage, size=len(sim_df))
    # adjust for mean_coverage
    simulated_coverage += mean_coverage - np.mean(simulated_coverage)
    # set coverage values smaller than min_coverage to min_coverage 
    simulated_coverage = np.maximum(simulated_coverage, min_coverage)


    # add return values to synthetic cohorts -----------------------------------
    sim_df["age"] = simulated_age
    sim_df["mean_age"] = np.mean(sim_df["age"])
    sim_df["coverage"] = simulated_coverage
    sim_df["mean_coverage"] = np.mean(sim_df["coverage"])
    sim_df["coverage"] = np.round(simulated_coverage).astype(int)


    # return result ------------------------------------------------------------
    return sim_df    



In [ ]:
# 2024_01_29: let's turn interactive code into a function
def sample_cohort_specs(n_donors = 250, mean_age = 50, sd_age = 10, 
                        min_age_bound = 20, max_age_bound = 90, 
                        intercept_coverage = 5000, slope_coverage = -400,
                        sd_coverage = 3000, min_coverage = 10,
                        max_coverage = 50000, mean_coverage=1000):#mean_coverage = 30000):
    """
    Generate synthetic cohort similar to GTEx data. The main point is to simulate
    coverage values which are linearly dependent on donor age. The parameter 
    values are chosen such that the properties of the simulated cohort match
    those of GTEx tissues best. These simulations are best thought of as "per 
    tissue, per position". 

    - donor ages are simulated using a truncated normal. 
    - coverage values are simulated using a linear model and Gaussian noise.
    

    n_donors:               number of donors to simulate 
    mean_age:               mean of the resulting age distribution
    sd_age:                 standard deviation of the resulting age distribution
    min_age_bound:          minimum of donor age distribution to simulate
    max_age_bound:          minimum of donor age distribution to simulate
    intercept_coverage:     coverage intercept of the age ~ coverage simulation. 
                            Values are learned from lm() coefs in real data.
    slope_coverage:         slope of the age ~ coverage simulation. lm() as above.
    mean_coverage:          mean of coverage values to aim for when simulating.
    sd_coverage:            std. deviation of coverage to aim for when simulating.
    min_coverage:           minimum coverage value found in real data.

    
    returns:   "synthetic cohort"  for tissue X and one position. Object is PD
               containing columns age, mean_age, coverage, mean_coverage, coverage.

    """

    # simulate cohort ----------------------------------------------------------
    # init empty data frame
    sim_df = pd.DataFrame(np.zeros(n_donors),columns=["donor_id"])
    # generate donor_id 
    sim_df['donor_id'] = 'donor_' + (sim_df.index + 1).astype(str)


    # simulate donor ages ------------------------------------------------------
    # calculate the truncated normal bounds
    a = (min_age - mean_age) / sd_age
    b = (max_age - mean_age) / sd_age
    # generate ages from a truncated normal distribution
    ages = truncnorm.rvs(a, b, loc = mean_age, scale = sd_age, size = n_donors)
    # round ages to integers
    simulated_age = np.round(ages).astype(int)


    # simulate coverage in linear dependence of donor age ----------------------
    # the key question here is how to relate mean_coverage to our linear model
    # simulated_coverage = intercep_coverage + simulated_age * slope_coverage + np.random.normal(loc=0, scale=sd_coverage, size=len(sim_df))
    simulated_coverage = simulated_age * slope_coverage + np.random.normal(loc=0, scale=sd_coverage, size=len(sim_df))
    
    # adjust for mean_coverage
    simulated_coverage += mean_coverage - np.mean(simulated_coverage)
    # set coverage values smaller than min_coverage to min_coverage 
    simulated_coverage = np.maximum(simulated_coverage, min_coverage)


    # add return values to synthetic cohorts -----------------------------------
    sim_df["age"] = simulated_age
    sim_df["mean_age"] = np.mean(sim_df["age"])
    sim_df["coverage"] = simulated_coverage
    sim_df["mean_coverage"] = np.mean(sim_df["coverage"])
    sim_df["coverage"] = np.round(simulated_coverage).astype(int)


    # return result ------------------------------------------------------------
    return sim_df    


In [ ]:
# 2024_01_29: --> archive this as previous: !!!! and do updates as discussed.

# def sample_cohort_specs(n_donors = 2000, min_age = 20, max_age = 90, n_pos = 200, 
#                        base_coverage = 10, slope = 0.3 , noise_factor = 0.01):
def sample_cohort_specs(n_donors = 2000, min_age = 20, max_age = 90, n_pos = 200, 
                        base_coverage = 10, intercept_factor=0.5, slope = 10, 
                        noise_factor = 15):
    """
    Sample age for n_pos and n_donors from normal dist bounded by min_age and 
    max_age. Generate coverage count values from a linear relationship between 
    coverage and age.

    Parameters:
    n_donors (int): number of donors for simulation cohort. 
    min_age (int): lower age bound for sampling donor age.
    max_age (int): upper age bound for sampling donor age.
    n_pos (int): number of positions to initialize per donor and for which coverage
    values will be sampled.
    effect_cov_age (int): number of coverage read counts which will be added per 
    age-year in the linear model for sampling coverage. 
    noise_factor (int): numeric factor indicating the amount of random noise added
    when simulation realtionship coverage with age.
    
    Returns:
    pd.DataFrame: sim_df representing a mock cohort with donor_ids, evenly spaced
    donor age, position_ids and coverage values which are positively correlated
    with age.
    """
    # init result df and aux params
    sim_df = pd.DataFrame(np.zeros(n_donors),columns=["donor_id"])

    # simulate donor age as evenly spaced values 
    age = np.linspace(min_age, max_age, n_donors)
    age = age.flatten()
    age = np.round(age).astype(int)
    sim_df["age"] = age

    # generate donor_id 
    sim_df['donor_id'] = 'donor_' + (sim_df.index + 1).astype(str)

    # init positions
    sim_df['pos_id'] = [list(range(1, 201)) for _ in range(len(sim_df))]
    sim_df = sim_df.explode('pos_id', ignore_index=True)

    # simulate coverage depending on age assuming positive linear relationship
    # coverage = sim_df["age"] * effect_cov_age + noise_factor * np.random.randn(len(sim_df)) - 900 # Paolo's way
    # coverage = 1000 + sim_df["age"] * effect_cov_age + noise_factor * np.random.randn(len(sim_df))
    # coverage = sim_df["age"] * effect_cov_age + noise_factor * np.random.randn(len(sim_df))  # let' use GTEx whole blood informed range of vals here  
    # coverage = sim_df["age"] + effect_cov_age + noise_factor * np.random.randn(len(sim_df))  # let' use GTEx whole blood informed range of vals here
                
    #coverage = np.random.normal(loc=base_coverage, scale=np.abs(sim_df['age'] * slope)) + noise_factor * np.random.randn(len(sim_df))  # let' use GTEx whole blood informed range of vals here  # SCALED COVERAGE!
    # coverage = base_coverage * slope + np.random.normal(loc=0,scale = slope)
    # coverage = 0 + sim_df['age'] *slope
    # coverage = 0 + sim_df['age'] * slope + np.random.normal(loc=0, scale=slope * noise_factor, size=len(sim_df))
    # coverage = base_coverage + sim_df['age'] * slope + np.random.normal(loc=0, scale=noise_factor, size=len(sim_df))
    
    # 2024_01_19: now flexibly adjust age coverage simulations proportional to specified values
    adjusted_intercept=base_coverage*intercept_factor
    adjusted_slope=(adjusted_intercept*slope)/base_coverage
    adjusted_noise=noise_factor*adjusted_intercept
    coverage = adjusted_intercept + sim_df['age'] * adjusted_slope + np.random.normal(loc=0, scale=adjusted_noise, size=len(sim_df))
    
    # Set negative values to zero
    coverage = np.maximum(coverage, 0) # 10 potentially
    # 2024_01_26: if < 0 do again or simulate more than you need and only take the ones above cutoff
    
    
    # sim_df["coverage"] = np.round(coverage).astype(int)
    # sim_df["coverage_scaled"] = coverage
    sim_df["coverage"] = coverage

    #return results    
    return sim_df
    

In [ ]:
params_df

## apply functions -- manual


In [ ]:
# set params manually for debugging before running automated pipeline above
#     Pos tissue                pval mean_coverage base_coverage  slope mean_het   var_het      fdr
# 1  1610 muscle_skeletal  0.00440            372.         559.   -3.72   0.139  0.0113    0.0396   
#intercept_factor=0.5
#base_coverage=4928
# noise_factor=0.01
#noise_factor = 0.05
#slope=-36.151260
mean_beta=0.027580
#var_beta=0.011053
var_beta=0.001337
# var_beta=0.0019020
# apply functions defined above
# simulate coverage 
#sim_df = sample_cohort_specs(base_coverage = base_coverage, intercept_factor=intercept_factor, slope = slope, noise_factor = noise_factor)

sim_list = []
# number of simulations
n_pos = 25
# loop through the simulation process n_pos times
for pos in range(1, n_pos + 1):
    # simulate coverage
    tmp_df = sample_cohort_specs()
    # simulate heteroplasmy inversely correlated with coverage using beta binomial
    tmp_df = sample_beta_binomial_fixed_variance(sim_df=tmp_df, mean_beta=mean_beta, var_beta=var_beta)
    # add Pos column 
    tmp_df["Pos"] = pos
    # append the result to the list
    sim_list.append(tmp_df)

# concatenate the results into a single DataFrame
sim_df = pd.concat(sim_list, ignore_index=True)

# show contents sim_df 
sim_df

In [ ]:
# plot heteroplasmy simultions
plt.figure(1, figsize=(18, 5))

# # 1
plt.subplot(131)
plt.plot(sim_df["age"], sim_df["coverage"], '.k')
plt.title('simulations: age vs. coverage', fontsize = 16, fontweight = "bold")
plt.xlabel('age [y]', fontsize = 14, fontweight = "bold")
plt.ylabel('coverage [reads]', fontsize = 14, fontweight = "bold")

# # 2
plt.subplot(132)
plt.plot(sim_df["age"], sim_df["het_level"], '.k')
plt.title('simulations: age vs. heteroplasmy', fontsize = 16, fontweight = "bold")
plt.ylabel('heteroplasmy [%]', fontsize = 14, fontweight = "bold")
plt.xlabel('age [y]', fontsize = 14, fontweight = "bold")
plt.ylim(0, 1)

# # 3
plt.subplot(133)
plt.plot(sim_df["coverage"], sim_df["het_level"], '.k')
plt.ylabel('heteteroplasmy [%]', fontsize = 14, fontweight = "bold")
plt.xlabel('coverage [reads]', fontsize = 14, fontweight = "bold")
plt.title('simulations: coverage vs. heteroplasmy', fontsize = 16, fontweight = "bold")
plt.ylim(0, 1)
plt.tight_layout()
plt.show()

In [ ]:
# save file to disc for running the models on it in separate `R` session 
# (they are all implemented in `R`, that's why...)
sim_df.to_csv("~/data/mtDNA_variants/metadata/cohort_files/gtex_v8/2023_12_13_test.tsv", sep="\t")
#sim_df.to_csv("~/data/mtDNA_variants/metadata/cohort_files/gtex_v8/2023_11_30_simulated_heteroplasmy_scaled_cov_slope_0_03_mean_het_0_5_scale_var_0_00000000000000000000000001.tsv", sep="\t")
#sim_df.to_csv("~/data/mtDNA_variants/metadata/cohort_files/gtex_v8/2023_11_30_simulated_heteroplasmy_age_on_cov_0_5_mean_het_0_5_scale_var_0_000005_lm_age_cov_and_overdisp.tsv", sep="\t")
#sim_df.to_csv("~/data/mtDNA_variants/metadata/cohort_files/gtex_v8/2023_11_28_simulated_heteroplasmy_mean_het_0_5_scale_var_0_15_no_cov_age_rel.tsv", sep="\t")

In [ ]:
# Create a new figure and set up the first subplot
plt.figure(figsize=(21, 5))

# hist age
plt.subplot(131)
plt.hist(sim_df["age"], bins=20, color='blue', alpha=0.7)
plt.title('Age Distribution', fontsize=16, fontweight="bold")
plt.xlabel('Age', fontsize=14, fontweight="bold")
plt.ylabel('Frequency', fontsize=14, fontweight="bold")

# hist coverage
plt.subplot(132)
plt.hist(sim_df["coverage"], bins=20, color='blue', alpha=0.7)
plt.title('Coverage Distribution', fontsize=16, fontweight="bold")
plt.xlabel('Coverage', fontsize=14, fontweight="bold")
plt.ylabel('Frequency', fontsize=14, fontweight="bold")


# Show the plot
plt.show()

In [ ]:
# SCRATCH ______________________________________________________________________

In [ ]:
def sample_beta_binomial(sim_df, mean_het = 0.5, scale_var = 0.05):
    """
    Sample heteroplasmic allele counts from a Beta-Binomial distribution. This 
    function is expected to be run on the outputs of sample_cohort_specs(). The 
    shape parameters of a Beta distribution are calculated from the desired mean
    heteroplasmy (desired in terms of simulation goal) and the variance for the 
    Beta. This variance is scaled such that it is inversely correlated with 
    coverage (overdispersion). The heteroplasmy level is returned as the fraction
    of simulated heteroplasmy counts over coverage.

    Parameters:
    sim_df (pd.DataFrame): simulated cohort data frame. Output of sample_cohort_specs().
    mean_het (int): The "true" mean heteroplasmy level we assume when simulating 
    overdispersion.
    scale_var (int): constant parameter which is used to scale the variance of the 
    Beta distribution inversely with coverage. 

    Returns:
    pd.DataFrame: sim_df is returned. Variance, alpha and beta shape parameters 
    for Beta distribution are returned. Also, p_beta, beta-estimated probability 
    for the Binomial draw to estimate heteroplasmy counts, and the heteroplasmy
    allele counts and levels are returned as well. 
    """
    sim_df["scale_var"] = scale_var
    sim_df["var_beta"] = (1/(sim_df["coverage"]*sim_df["scale_var"]))

    # set mean to expected mean_het
    sim_df["mean_het"] = mean_het

    # calculate alpha and beta from mean and variance
    sim_df["common_factor"] = sim_df["mean_het"] * (1 - sim_df["mean_het"]) / sim_df["var_beta"] - 1
    sim_df["alpha"] = sim_df["mean_het"] * sim_df["common_factor"]
    sim_df["beta"] = (1 - sim_df["mean_het"]) * sim_df["common_factor"]

    # sample from p_beta from Beta for using it in Bernoulli draw below
    sim_df["p_beta"] = np.random.beta(sim_df["alpha"],sim_df["beta"])

    # sample from the Binomial distribution using the sampled Beta value
    sim_df["het_counts"] = np.random.binomial(sim_df["coverage"], sim_df["p_beta"])
    sim_df["het_level"] = sim_df["het_counts"]/sim_df["coverage"]
        
    return sim_df       

In [ ]:
mean_beta=0.5
var_beta=0.1
expand_coverage_by=1000


common_factor = sim_df["mean_het"] * (1 - sim_df["mean_het"]) / sim_df["var_beta"] - 1
alpha = sim_df["mean_het"] * sim_df["common_factor"]
beta = (1 - sim_df["mean_het"]) * sim_df["common_factor"]

# sample from p_beta from Beta for using it in Bernoulli draw below
p_beta = np.random.beta(sim_df["alpha"],sim_df["beta"])

# artifically expand_coverage from decimal values to integers since we 
# need integer counts for beta binomial regression 
coverage = sim_df["coverage_scaled"] * expand_coverage_by
coverage = np.round(coverage).astype(int)

het_counts = np.random.binomial(coverage, sim_df["p_beta"])
het_level = sim_df["het_counts"]/coverage

het_level

In [ ]:
# apply functions defined above

# simulate mock cohort with linear age-coverage relationship
# sim_df = sample_cohort_specs(effect_cov_age=0,noise_factor=120)
# sim_df = sample_cohort_specs(effect_cov_age = 50, noise_factor = 100)
# sim_df = sample_cohort_specs(effect_cov_age = 0.5, noise_factor = 2.5)
# sim_df = sample_cohort_specs(base_coverage = 20, slope = 0.04 , noise_factor = 0.5)

sim_df = sample_cohort_specs(base_coverage = 10, slope = 5, noise_factor = 15)

# simulate heteroplasmy inversely correlated with coverage using beta binomial
# sim_df = sample_beta_binomial(sim_df=sim_df, mean_het=0.2, scale_var=0.15)
# sim_df = sample_beta_binomial(sim_df=sim_df, mean_het=0.5, scale_var=0.15)

# simulate heteroplasmy not correlated with coverage using beta binomial with 
# constant variance (just for checking this)
# sim_df = sample_beta_binomial_fixed_variance(sim_df=sim_df, mean_beta=0.5, var_beta=0.00000000000000000000000001)
# sim_df = sample_beta_binomial_fixed_variance(sim_df=sim_df, mean_beta=0.5, var_beta=0.000005)


sim_df = sample_beta_binomial_fixed_variance(sim_df=sim_df, mean_beta=0.1, var_beta=0.0000005)


# show contents sim_df 
sim_df